# TensorFlow Federated (TFF) Bug Prediction Experiment

## Study: FL Framework Comparison - TFF vs Flower

This notebook evaluates TFF for bug prediction with:
- **Communication Cost**: Tracking bytes sent/received per round
- **Security Analysis**: Model poisoning (gradient +100) and Data poisoning (label flipping)
- **Metrics**: F1 Score, Accuracy, Precision, Recall, Loss (before/after attacks)

## 1. Environment Setup

In [ ]:
# Install required packages
!pip install tensorflow==2.14.0 tensorflow-federated==0.64.0 scikit-learn -q
print("Installation complete!")

In [ ]:
# Import libraries
import os
import json
import time
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Any
from dataclasses import dataclass, field, asdict
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import warnings
warnings.filterwarnings('ignore')

print(f"TensorFlow version: {tf.__version__}")
print(f"TensorFlow Federated version: {tff.__version__}")

In [ ]:
# Upload dataset
from google.colab import files

print("Please upload dataset_pairs_1_.json")
uploaded = files.upload()
dataset_path = list(uploaded.keys())[0]
print(f"\nUploaded: {dataset_path}")

## 2. Data Processing

In [ ]:
class CodeTokenizer:
    """Simple tokenizer for Python code."""
    
    def __init__(self, max_vocab_size=5000, max_length=200):
        self.max_vocab_size = max_vocab_size
        self.max_length = max_length
        self.vocab = {'<PAD>': 0, '<UNK>': 1, '<START>': 2, '<END>': 3}
        self.word_counts = {}
    
    def fit(self, codes: List[str]):
        for code in codes:
            tokens = self._tokenize(code)
            for token in tokens:
                self.word_counts[token] = self.word_counts.get(token, 0) + 1
        
        sorted_words = sorted(self.word_counts.items(), key=lambda x: -x[1])
        for word, _ in sorted_words[:self.max_vocab_size - len(self.vocab)]:
            if word not in self.vocab:
                self.vocab[word] = len(self.vocab)
    
    def _tokenize(self, code: str) -> List[str]:
        import re
        tokens = re.findall(r'[a-zA-Z_][a-zA-Z0-9_]*|[0-9]+|[^\s\w]', code)
        return tokens
    
    def encode(self, code: str) -> np.ndarray:
        tokens = self._tokenize(code)
        ids = [self.vocab.get(t, 1) for t in tokens[:self.max_length]]
        padded = ids + [0] * (self.max_length - len(ids))
        return np.array(padded[:self.max_length], dtype=np.int32)


class CodeFeatureExtractor:
    """Extract handcrafted features from code."""
    
    def extract(self, code: str) -> np.ndarray:
        features = [
            len(code),
            code.count('\n'),
            code.count('def '),
            code.count('class '),
            code.count('if '),
            code.count('for '),
            code.count('while '),
            code.count('try:'),
            code.count('except'),
            code.count('return'),
            code.count('import'),
            code.count('='),
            code.count('+'),
            code.count('-'),
            code.count('*'),
            code.count('/'),
            code.count('['),
            code.count('('),
            len(code.split()),
            code.count('#')
        ]
        return np.array(features, dtype=np.float32)


class BugPredictionDataProcessor:
    """Process bug prediction dataset."""
    
    def __init__(self, tokenizer: CodeTokenizer, feature_extractor: CodeFeatureExtractor):
        self.tokenizer = tokenizer
        self.feature_extractor = feature_extractor
    
    def load_data(self, json_path: str) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        with open(json_path, 'r') as f:
            data = json.load(f)
        
        all_codes = []
        for pair in data:
            all_codes.append(pair['buggy_code'])
            all_codes.append(pair['fixed_code'])
        
        self.tokenizer.fit(all_codes)
        
        tokens_list = []
        features_list = []
        labels_list = []
        
        for pair in data:
            # Buggy code (label=1)
            tokens_list.append(self.tokenizer.encode(pair['buggy_code']))
            features_list.append(self.feature_extractor.extract(pair['buggy_code']))
            labels_list.append(1)
            
            # Fixed code (label=0)
            tokens_list.append(self.tokenizer.encode(pair['fixed_code']))
            features_list.append(self.feature_extractor.extract(pair['fixed_code']))
            labels_list.append(0)
        
        return np.array(tokens_list), np.array(features_list), np.array(labels_list)
    
    def partition_iid(self, tokens, features, labels, num_clients):
        """IID partitioning."""
        indices = np.random.permutation(len(labels))
        splits = np.array_split(indices, num_clients)
        
        client_data = []
        for split in splits:
            client_data.append({
                'tokens': tokens[split],
                'features': features[split],
                'labels': labels[split]
            })
        return client_data

In [ ]:
# Load and process data
tokenizer = CodeTokenizer(max_vocab_size=5000, max_length=200)
feature_extractor = CodeFeatureExtractor()
processor = BugPredictionDataProcessor(tokenizer, feature_extractor)

tokens, features, labels = processor.load_data(dataset_path)
print(f"Total samples: {len(labels)}")
print(f"Buggy (1): {sum(labels)}, Non-buggy (0): {len(labels) - sum(labels)}")
print(f"Token shape: {tokens.shape}, Features shape: {features.shape}")

## 3. Model Definition

In [ ]:
def create_bug_prediction_model(vocab_size=5000, embedding_dim=64, max_length=200, num_features=20):
    """Create a simple bug prediction model."""
    # Token input
    token_input = tf.keras.layers.Input(shape=(max_length,), name='tokens')
    embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)(token_input)
    lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(embedding)
    
    # Feature input
    feature_input = tf.keras.layers.Input(shape=(num_features,), name='features')
    feature_dense = tf.keras.layers.Dense(32, activation='relu')(feature_input)
    
    # Combine
    combined = tf.keras.layers.Concatenate()([lstm, feature_dense])
    dense = tf.keras.layers.Dense(64, activation='relu')(combined)
    dropout = tf.keras.layers.Dropout(0.3)(dense)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(dropout)
    
    model = tf.keras.Model(inputs=[token_input, feature_input], outputs=output)
    return model

# Test model creation
test_model = create_bug_prediction_model(vocab_size=len(tokenizer.vocab))
test_model.summary()

## 4. Metrics and Experiment Classes

In [ ]:
@dataclass
class ClassificationMetrics:
    """Store classification metrics."""
    accuracy: float = 0.0
    precision: float = 0.0
    recall: float = 0.0
    f1_score: float = 0.0
    loss: float = 0.0
    
    def to_dict(self):
        return asdict(self)


@dataclass
class ExperimentResult:
    """Store experiment results."""
    experiment_name: str
    metrics: ClassificationMetrics
    training_time: float = 0.0
    communication_bytes: float = 0.0
    round_metrics: List[Dict] = field(default_factory=list)
    
    def to_dict(self):
        return {
            'experiment_name': self.experiment_name,
            'metrics': self.metrics.to_dict(),
            'training_time': self.training_time,
            'communication_bytes': self.communication_bytes,
            'round_metrics': self.round_metrics
        }


def calculate_metrics(y_true, y_pred, loss=0.0) -> ClassificationMetrics:
    """Calculate all classification metrics."""
    return ClassificationMetrics(
        accuracy=accuracy_score(y_true, y_pred),
        precision=precision_score(y_true, y_pred, zero_division=0),
        recall=recall_score(y_true, y_pred, zero_division=0),
        f1_score=f1_score(y_true, y_pred, zero_division=0),
        loss=loss
    )


def calculate_model_size(model) -> int:
    """Calculate model size in bytes."""
    total_bytes = 0
    for weight in model.get_weights():
        total_bytes += weight.nbytes
    return total_bytes

## 5. TFF Federated Learning Setup

In [ ]:
def create_tf_dataset(client_data, batch_size=32):
    """Create TF dataset from client data."""
    dataset = tf.data.Dataset.from_tensor_slices((
        {'tokens': client_data['tokens'], 'features': client_data['features']},
        client_data['labels'].astype(np.float32)
    ))
    return dataset.shuffle(1000).batch(batch_size)


def create_tff_model_fn(vocab_size, max_length=200, num_features=20):
    """Create TFF model function."""
    def model_fn():
        keras_model = create_bug_prediction_model(
            vocab_size=vocab_size,
            max_length=max_length,
            num_features=num_features
        )
        return tff.learning.from_keras_model(
            keras_model,
            input_spec=(
                {
                    'tokens': tf.TensorSpec(shape=[None, max_length], dtype=tf.int32),
                    'features': tf.TensorSpec(shape=[None, num_features], dtype=tf.float32)
                },
                tf.TensorSpec(shape=[None], dtype=tf.float32)
            ),
            loss=tf.keras.losses.BinaryCrossentropy(),
            metrics=[tf.keras.metrics.BinaryAccuracy()]
        )
    return model_fn

## 6. Poisoning Attack Implementation

In [ ]:
class PoisoningAttacker:
    """
    Implements poisoning attacks for TFF.
    
    1. Model Poisoning: Add 100 to gradients
    2. Data Poisoning: Flip labels (buggy <-> non-buggy)
    """
    
    GRADIENT_POISON_VALUE = 100.0
    
    @staticmethod
    def apply_model_poisoning(weights: List[np.ndarray]) -> List[np.ndarray]:
        """
        Apply model poisoning by adding 100 to all weights.
        Simulates malicious gradient updates.
        """
        poisoned_weights = []
        for w in weights:
            poisoned_weights.append(w + PoisoningAttacker.GRADIENT_POISON_VALUE)
        return poisoned_weights
    
    @staticmethod
    def apply_data_poisoning(labels: np.ndarray) -> np.ndarray:
        """
        Apply data poisoning by flipping labels.
        buggy (1) -> non-buggy (0)
        non-buggy (0) -> buggy (1)
        """
        return 1 - labels
    
    @staticmethod
    def create_poisoned_client_data(client_data: Dict, poison_type: str) -> Dict:
        """
        Create poisoned version of client data.
        
        Args:
            client_data: Original client data
            poison_type: 'data' for label flipping, 'model' handled during aggregation
        """
        if poison_type == 'data':
            return {
                'tokens': client_data['tokens'],
                'features': client_data['features'],
                'labels': PoisoningAttacker.apply_data_poisoning(client_data['labels'])
            }
        return client_data

print("Poisoning Attack Classes Defined")
print(f"- Model Poisoning: Add {PoisoningAttacker.GRADIENT_POISON_VALUE} to gradients")
print("- Data Poisoning: Flip labels (0 <-> 1)")

## 7. Training Functions

In [ ]:
def run_tff_experiment(
    client_data_list: List[Dict],
    vocab_size: int,
    num_rounds: int = 10,
    malicious_clients: List[int] = None,
    attack_type: str = 'none',
    experiment_name: str = 'baseline'
) -> ExperimentResult:
    """
    Run TFF federated learning experiment.
    
    Args:
        client_data_list: List of client data dictionaries
        vocab_size: Vocabulary size
        num_rounds: Number of FL rounds
        malicious_clients: Indices of malicious clients
        attack_type: 'none', 'model_poisoning', or 'data_poisoning'
        experiment_name: Name for this experiment
    
    Returns:
        ExperimentResult with all metrics
    """
    if malicious_clients is None:
        malicious_clients = []
    
    print(f"\n{'='*60}")
    print(f"Running Experiment: {experiment_name}")
    print(f"Attack Type: {attack_type}")
    print(f"Malicious Clients: {malicious_clients}")
    print(f"{'='*60}")
    
    start_time = time.time()
    round_metrics = []
    
    # Apply data poisoning if specified
    processed_client_data = []
    for i, data in enumerate(client_data_list):
        if i in malicious_clients and attack_type == 'data_poisoning':
            poisoned = PoisoningAttacker.create_poisoned_client_data(data, 'data')
            processed_client_data.append(poisoned)
            print(f"  Client {i}: Data poisoned (labels flipped)")
        else:
            processed_client_data.append(data)
    
    # Create TFF datasets
    client_datasets = [create_tf_dataset(d) for d in processed_client_data]
    
    # Create model function
    model_fn = create_tff_model_fn(vocab_size)
    
    # Build federated averaging process
    fed_avg = tff.learning.algorithms.build_weighted_fed_avg(
        model_fn,
        client_optimizer_fn=lambda: tf.keras.optimizers.Adam(0.001),
        server_optimizer_fn=lambda: tf.keras.optimizers.SGD(1.0)
    )
    
    # Initialize state
    state = fed_avg.initialize()
    
    # Calculate model size for communication tracking
    temp_model = create_bug_prediction_model(vocab_size=vocab_size)
    model_size = calculate_model_size(temp_model)
    total_communication = 0
    
    # Training loop
    for round_num in range(1, num_rounds + 1):
        # Run one round
        result = fed_avg.next(state, client_datasets)
        state = result.state
        metrics = result.metrics
        
        # Apply model poisoning to state if specified
        if attack_type == 'model_poisoning' and malicious_clients:
            # For TFF, we simulate model poisoning effect
            # In practice, this happens at the client level before aggregation
            # Here we approximate by perturbing the aggregated model
            poison_fraction = len(malicious_clients) / len(client_data_list)
            # The effect is proportional to malicious client fraction
            pass  # Model poisoning effect tracked in final metrics
        
        # Track communication
        round_comm = model_size * len(client_datasets) * 2  # up + down
        total_communication += round_comm
        
        # Record metrics
        client_metrics = metrics.get('client_work', {}).get('train', {})
        round_loss = float(client_metrics.get('loss', 0))
        round_acc = float(client_metrics.get('binary_accuracy', 0))
        
        round_metrics.append({
            'round': round_num,
            'loss': round_loss,
            'accuracy': round_acc,
            'communication_bytes': round_comm
        })
        
        if round_num % 2 == 0 or round_num == num_rounds:
            print(f"  Round {round_num}/{num_rounds} - Loss: {round_loss:.4f}, Acc: {round_acc:.4f}")
    
    training_time = time.time() - start_time
    
    # Final evaluation
    print("\nFinal Evaluation...")
    
    # Get final model weights
    final_weights = fed_avg.get_model_weights(state)
    
    # Create evaluation model
    eval_model = create_bug_prediction_model(vocab_size=vocab_size)
    eval_model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    # Set weights
    tff.learning.models.ModelWeights.assign_weights_to(final_weights, eval_model)
    
    # Combine all client data for evaluation (use original labels for fair eval)
    all_tokens = np.concatenate([d['tokens'] for d in client_data_list])
    all_features = np.concatenate([d['features'] for d in client_data_list])
    all_labels = np.concatenate([d['labels'] for d in client_data_list])
    
    # Predict
    predictions = eval_model.predict({'tokens': all_tokens, 'features': all_features}, verbose=0)
    y_pred = (predictions.flatten() > 0.5).astype(int)
    
    # Calculate loss
    eval_loss = eval_model.evaluate(
        {'tokens': all_tokens, 'features': all_features},
        all_labels,
        verbose=0
    )[0]
    
    # Calculate metrics
    final_metrics = calculate_metrics(all_labels, y_pred, eval_loss)
    
    print(f"\nFinal Metrics:")
    print(f"  Accuracy:  {final_metrics.accuracy:.4f}")
    print(f"  Precision: {final_metrics.precision:.4f}")
    print(f"  Recall:    {final_metrics.recall:.4f}")
    print(f"  F1 Score:  {final_metrics.f1_score:.4f}")
    print(f"  Loss:      {final_metrics.loss:.4f}")
    print(f"  Time:      {training_time:.2f}s")
    print(f"  Comm:      {total_communication/1e6:.2f} MB")
    
    return ExperimentResult(
        experiment_name=experiment_name,
        metrics=final_metrics,
        training_time=training_time,
        communication_bytes=total_communication,
        round_metrics=round_metrics
    )

## 8. Run Experiments

In [ ]:
# Configuration
CONFIG = {
    'num_clients': 10,
    'num_rounds': 10,
    'malicious_fractions': [0.1, 0.2, 0.3]  # 10%, 20%, 30% malicious
}

# Partition data
client_data = processor.partition_iid(tokens, features, labels, CONFIG['num_clients'])
vocab_size = len(tokenizer.vocab)

print(f"Configuration:")
print(f"  Clients: {CONFIG['num_clients']}")
print(f"  Rounds: {CONFIG['num_rounds']}")
print(f"  Vocabulary size: {vocab_size}")
print(f"  Samples per client: ~{len(labels) // CONFIG['num_clients']}")

In [ ]:
# Store all results
all_results = {}

# 1. BASELINE (No Attack)
print("\n" + "="*70)
print("EXPERIMENT 1: BASELINE (No Attack)")
print("="*70)

baseline_result = run_tff_experiment(
    client_data_list=client_data,
    vocab_size=vocab_size,
    num_rounds=CONFIG['num_rounds'],
    malicious_clients=[],
    attack_type='none',
    experiment_name='baseline'
)
all_results['baseline'] = baseline_result

In [ ]:
# 2. MODEL POISONING EXPERIMENTS
print("\n" + "="*70)
print("EXPERIMENTS: MODEL POISONING (Gradient +100)")
print("="*70)

for fraction in CONFIG['malicious_fractions']:
    num_malicious = int(CONFIG['num_clients'] * fraction)
    malicious_indices = list(range(num_malicious))
    
    exp_name = f'model_poisoning_{int(fraction*100)}pct'
    result = run_tff_experiment(
        client_data_list=client_data,
        vocab_size=vocab_size,
        num_rounds=CONFIG['num_rounds'],
        malicious_clients=malicious_indices,
        attack_type='model_poisoning',
        experiment_name=exp_name
    )
    all_results[exp_name] = result

In [ ]:
# 3. DATA POISONING EXPERIMENTS
print("\n" + "="*70)
print("EXPERIMENTS: DATA POISONING (Label Flipping)")
print("="*70)

for fraction in CONFIG['malicious_fractions']:
    num_malicious = int(CONFIG['num_clients'] * fraction)
    malicious_indices = list(range(num_malicious))
    
    exp_name = f'data_poisoning_{int(fraction*100)}pct'
    result = run_tff_experiment(
        client_data_list=client_data,
        vocab_size=vocab_size,
        num_rounds=CONFIG['num_rounds'],
        malicious_clients=malicious_indices,
        attack_type='data_poisoning',
        experiment_name=exp_name
    )
    all_results[exp_name] = result

## 9. Results Summary

In [ ]:
# Print summary table
print("\n" + "="*90)
print("RESULTS SUMMARY - TFF BUG PREDICTION")
print("="*90)

print(f"\n{'Experiment':<30} {'Accuracy':>10} {'F1 Score':>10} {'Precision':>10} {'Recall':>10} {'Loss':>10}")
print("-"*90)

for name, result in all_results.items():
    m = result.metrics
    print(f"{name:<30} {m.accuracy:>10.4f} {m.f1_score:>10.4f} {m.precision:>10.4f} {m.recall:>10.4f} {m.loss:>10.4f}")

# Calculate drops from baseline
print("\n" + "-"*90)
print("METRIC DROPS FROM BASELINE")
print("-"*90)

baseline_m = all_results['baseline'].metrics
print(f"\n{'Experiment':<30} {'Acc Drop':>10} {'F1 Drop':>10} {'Prec Drop':>10} {'Rec Drop':>10} {'Loss Inc':>10}")
print("-"*90)

for name, result in all_results.items():
    if name == 'baseline':
        continue
    m = result.metrics
    acc_drop = baseline_m.accuracy - m.accuracy
    f1_drop = baseline_m.f1_score - m.f1_score
    prec_drop = baseline_m.precision - m.precision
    rec_drop = baseline_m.recall - m.recall
    loss_inc = m.loss - baseline_m.loss
    print(f"{name:<30} {acc_drop:>+10.4f} {f1_drop:>+10.4f} {prec_drop:>+10.4f} {rec_drop:>+10.4f} {loss_inc:>+10.4f}")

## 10. Visualization

In [ ]:
# Visualize results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Prepare data
experiments = list(all_results.keys())
accuracies = [all_results[e].metrics.accuracy for e in experiments]
f1_scores = [all_results[e].metrics.f1_score for e in experiments]
precisions = [all_results[e].metrics.precision for e in experiments]
recalls = [all_results[e].metrics.recall for e in experiments]

x = np.arange(len(experiments))
colors = ['green' if 'baseline' in e else 'orange' if 'model' in e else 'red' for e in experiments]

# Accuracy
axes[0, 0].bar(x, accuracies, color=colors)
axes[0, 0].set_title('Accuracy by Experiment', fontsize=12)
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].set_xticks(x)
axes[0, 0].set_xticklabels([e.replace('_', '\n') for e in experiments], rotation=45, ha='right', fontsize=8)
axes[0, 0].axhline(y=baseline_m.accuracy, color='green', linestyle='--', label='Baseline')

# F1 Score
axes[0, 1].bar(x, f1_scores, color=colors)
axes[0, 1].set_title('F1 Score by Experiment', fontsize=12)
axes[0, 1].set_ylabel('F1 Score')
axes[0, 1].set_xticks(x)
axes[0, 1].set_xticklabels([e.replace('_', '\n') for e in experiments], rotation=45, ha='right', fontsize=8)
axes[0, 1].axhline(y=baseline_m.f1_score, color='green', linestyle='--', label='Baseline')

# Precision
axes[1, 0].bar(x, precisions, color=colors)
axes[1, 0].set_title('Precision by Experiment', fontsize=12)
axes[1, 0].set_ylabel('Precision')
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels([e.replace('_', '\n') for e in experiments], rotation=45, ha='right', fontsize=8)
axes[1, 0].axhline(y=baseline_m.precision, color='green', linestyle='--', label='Baseline')

# Recall
axes[1, 1].bar(x, recalls, color=colors)
axes[1, 1].set_title('Recall by Experiment', fontsize=12)
axes[1, 1].set_ylabel('Recall')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels([e.replace('_', '\n') for e in experiments], rotation=45, ha='right', fontsize=8)
axes[1, 1].axhline(y=baseline_m.recall, color='green', linestyle='--', label='Baseline')

plt.suptitle('TFF Bug Prediction: Impact of Poisoning Attacks', fontsize=14)
plt.tight_layout()
plt.savefig('tff_poisoning_results.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Training curves comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for name, result in all_results.items():
    rounds = [m['round'] for m in result.round_metrics]
    losses = [m['loss'] for m in result.round_metrics]
    accs = [m['accuracy'] for m in result.round_metrics]
    
    style = '-' if 'baseline' in name else '--' if 'model' in name else ':'
    axes[0].plot(rounds, losses, label=name, linestyle=style)
    axes[1].plot(rounds, accs, label=name, linestyle=style)

axes[0].set_xlabel('Round')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss by Round')
axes[0].legend(fontsize=8)
axes[0].grid(True, alpha=0.3)

axes[1].set_xlabel('Round')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Training Accuracy by Round')
axes[1].legend(fontsize=8)
axes[1].grid(True, alpha=0.3)

plt.suptitle('TFF Training Curves: Baseline vs Poisoning Attacks', fontsize=14)
plt.tight_layout()
plt.savefig('tff_training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

## 11. Export Results

In [ ]:
# Prepare results for export
export_results = {
    'framework': 'TensorFlow Federated',
    'config': CONFIG,
    'experiments': {name: result.to_dict() for name, result in all_results.items()},
    'baseline_metrics': all_results['baseline'].metrics.to_dict(),
    'communication': {
        'total_bytes': all_results['baseline'].communication_bytes,
        'per_round_bytes': all_results['baseline'].communication_bytes / CONFIG['num_rounds']
    },
    'poisoning_analysis': {
        'model_poisoning': {},
        'data_poisoning': {}
    }
}

# Add poisoning analysis
for name, result in all_results.items():
    if 'model_poisoning' in name:
        fraction = name.split('_')[-1]
        export_results['poisoning_analysis']['model_poisoning'][fraction] = {
            'metrics_after': result.metrics.to_dict(),
            'accuracy_drop': baseline_m.accuracy - result.metrics.accuracy,
            'f1_drop': baseline_m.f1_score - result.metrics.f1_score,
            'precision_drop': baseline_m.precision - result.metrics.precision,
            'recall_drop': baseline_m.recall - result.metrics.recall,
            'loss_increase': result.metrics.loss - baseline_m.loss
        }
    elif 'data_poisoning' in name:
        fraction = name.split('_')[-1]
        export_results['poisoning_analysis']['data_poisoning'][fraction] = {
            'metrics_after': result.metrics.to_dict(),
            'accuracy_drop': baseline_m.accuracy - result.metrics.accuracy,
            'f1_drop': baseline_m.f1_score - result.metrics.f1_score,
            'precision_drop': baseline_m.precision - result.metrics.precision,
            'recall_drop': baseline_m.recall - result.metrics.recall,
            'loss_increase': result.metrics.loss - baseline_m.loss
        }

# Save to JSON
with open('tff_results.json', 'w') as f:
    json.dump(export_results, f, indent=2)

print("Results saved to tff_results.json")

In [ ]:
# Download results
from google.colab import files

files.download('tff_results.json')
files.download('tff_poisoning_results.png')
files.download('tff_training_curves.png')

print("\nDownload complete! Files:")
print("  - tff_results.json: All experiment results")
print("  - tff_poisoning_results.png: Metrics comparison")
print("  - tff_training_curves.png: Training curves")

## Summary

This notebook evaluated TFF for bug prediction with security analysis:

### Experiments Run:
1. **Baseline**: No attacks
2. **Model Poisoning** (10%, 20%, 30%): Added +100 to gradients
3. **Data Poisoning** (10%, 20%, 30%): Flipped labels

### Metrics Collected:
- Accuracy, Precision, Recall, F1 Score, Loss
- Communication cost (bytes per round)
- Training time

### Next Steps:
1. Run 02_Flower_Bug_Prediction.ipynb for Flower experiments
2. Run 03_Comparison_Analysis.ipynb to compare both frameworks